In [1]:
using Plots

In [45]:
include("himalayan_option.jl")

himalayan_option_monte_carlo (generic function with 1 method)

In [35]:
function compare_variance(num_of_sim::Int, α::Float64,r::Float64, basket_volume::Int, S₀::Array{Float64}, K::Float64, mu::Array{Float64}, 
    sigma::Array{Float64}, correlation_matrix::Matrix{Float64})

    normal = himalayan_option_monte_carlo(num_of_sim, α, r, basket_volume, S₀, K, mu, sigma, correlation_matrix,"basic")
    antithetic = himalayan_option_monte_carlo(num_of_sim, α, r, basket_volume, S₀, K, mu, sigma, correlation_matrix,"antithetic")
    quasi = himalayan_option_monte_carlo(num_of_sim, α, r, basket_volume, S₀, K, mu, sigma, correlation_matrix,"quasi_monte_carlo")
    moment_matching = price_himalayan_moment_matching(num_of_sim, α, r, basket_volume, S₀, K, mu, sigma, correlation_matrix)
    LHS = himalayan_option_monte_carlo(num_of_sim, α, r, basket_volume, S₀, K, mu, sigma, correlation_matrix,"LHS") 
     
    mean_elements = normal[1]
    bottom_elements = normal[2]
    upper_elements = normal[3]

    mean_antithetic = antithetic[1]
    bottom_antithetic = antithetic[2]
    upper_antithetic = antithetic[3]

    mean_quasi = quasi

    mean_moment_matching = moment_matching[1]
    bottom_moment_matching = moment_matching[2]
    upper_moment_matching = moment_matching[3]

    mean_LHS = LHS[1]
    bottom_LHS = LHS[2]
    upper_LHS = LHS[3]

    length_normal = upper_elements-bottom_elements
    length_antithetic = upper_antithetic - bottom_antithetic
    length_moment_matching = upper_moment_matching - bottom_moment_matching
    length_LHS = upper_LHS-bottom_LHS

    return [mean_elements, mean_antithetic, mean_moment_matching, mean_LHS, mean_quasi, 
            length_normal, length_antithetic, length_moment_matching, length_LHS]
end

function num_of_sims(num_of_sims::Array{Int}, α::Float64,r::Float64, basket_volume::Int, S₀::Array{Float64}, K::Float64, mu::Array{Float64}, 
    sigma::Array{Float64}, correlation_matrix::Matrix{Float64})

    results::Matrix{Float64} = zeros(length(num_of_sims), 9)
    for (ind, num_of_sim) in enumerate(num_of_sims)
        results[ind,:] = compare_variance(num_of_sim, α, r, basket_volume, S₀, K, mu, sigma, correlation_matrix)
    end

    return results

end 

function mu_value(num_of_sim::Int, α::Float64,r::Float64, basket_volume::Int, S₀::Array{Float64}, K::Float64, mu::Array{Float64}, 
    sigma::Array{Float64}, correlation_matrix::Matrix{Float64})


    results::Matrix{Float64} = zeros(length(mu), 9)
    for (ind, m) in enumerate(mu)
        results[ind,:] = compare_variance(num_of_sim, α, r, basket_volume, S₀, K, [m for i in 1:basket_volume], sigma, correlation_matrix)
    end

    return results

end

function interest_rate(num_of_sim::Int, α::Float64,rs::Array{Float64}, basket_volume::Int, S₀::Array{Float64}, K::Float64, mu::Array{Float64}, 
                    sigma::Array{Float64}, correlation_matrix::Matrix{Float64})


    results::Matrix{Float64} = zeros(length(rs), 9)
    for (ind, r) in enumerate(rs)
        results[ind,:] = compare_variance(num_of_sim, α, r, basket_volume, S₀, K, mu, sigma, correlation_matrix)
    end

    return results

end 


function correlation(num_of_sim::Int, α::Float64,r::Float64, basket_volume::Int, S₀::Array{Float64}, K::Float64, mu::Array{Float64}, 
    sigma::Array{Float64})

    corr_matrices::Vector{Matrix{Float64}} = [[1.0 0.1 0.1; 0.1 1.0 0.1; 0.1 0.1 1.0], [1.0 0.5 0.4; 0.5 1.0 0.6; 0.4 0.6 1.0], [1.0 0.9 0.8; 0.9 1.0 0.7; 0.8 0.7 1.0]] 

    results::Matrix{Float64} = zeros(3, 9)
    for correlation in eachindex(corr_matrices)
        results[correlation,:] = compare_variance(num_of_sim, α, r, basket_volume, S₀, K, mu, sigma, corr_matrices[correlation])
    end
    return results

end

function plot_variance(vals, variance, title, xlabel)
    scatter(vals,variance, label = ["normal" "antithetic" "moment_matching" "LHS"], title = title, legend = :outertop)
    xlabel!(xlabel)
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\himalayan\\", title, "_himalayan_", ".svg"))
end

function plot_mean(vals, mean, title, xlabel)
    scatter(vals, mean, label=["mean_elements" "mean_antithetic" "mean_moment_matching" "mean_LHS" "mean_quasi"], legend = :outertop)
    title!(title)
    xlabel!(xlabel)
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\himalayan\\", title,"_himalayan_", ".svg"))
end

function plot_bar_plot(results,name)
    bar(["słaba", "średnia", "silna"],results,label=name,title=string("długość przedziału ~ korelacja dla metody ",name), xlabel = "siła korelacji między spółkami", xticks = :all,legend=false, fmt = :svg)    
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\himalayan\\", name, "_himalayan_", ".svg"))
end


plot_bar_plot (generic function with 1 method)

In [36]:
cov_matrix = [1.0 0.3 0.4; 0.3 1.0 0.1; 0.4 0.1 1.0] 

3×3 Matrix{Float64}:
 1.0  0.3  0.4
 0.3  1.0  0.1
 0.4  0.1  1.0

# Liczba symulacji a szerokość przedziału

In [46]:
n_of_sims = [100:100:2000;]
sims::Matrix{Float64} = num_of_sims(n_of_sims,0.05,0.04,3,[10.0,10.0,10.0], 11.0, [0.07,0.07,0.07],[0.2,0.2,0.2], cov_matrix)
variance = sims[:,6:end]
plot_variance(n_of_sims,variance,"długość przedziału ufności ~ wielkość próby", "wielkość próby")
means = sims[:,1:5]
plot_mean(n_of_sims,means,"cena opcji ~ wielkość próby", "wielkość próby")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\himalayan\\cena opcji ~ wielkość próby_himalayan_.svg"

# Variance ~ trend

In [50]:
mu = [0.01:0.01:0.2;]
sims_mu::Matrix{Float64} = mu_value(1000,0.05,0.04,3,[10.0,10.0,10.0], 9.0, mu, [0.2,0.2,0.2], cov_matrix)
variance_mu = sims_mu[:,6:end]
plot_variance(mu,variance_mu, "długość przedziału ufności ~ trend", "trend")
means_mu = sims_mu[:,1:5]
plot_mean(mu,means_mu, "cena opcji ~ trend", "trend")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\himalayan\\cena opcji ~ trend_himalayan_.svg"

# Stopy procentowe

In [51]:
rs = [0.01:0.001:0.1;]
sims_rs::Matrix{Float64} = interest_rate(1000,0.05,rs,3,[10.0,10.0,10.0], 9.0, [0.07,0.07,0.07],[0.2,0.2,0.2], cov_matrix)
variance_rs = sims_rs[:,6:end]
plot_variance(rs,variance_rs, "długość przedziału ufności ~ stopy procentowe", "stopy procentowe")
means_rs = sims_rs[:,1:5]
plot_mean(rs,means_rs, "cena opcji ~ stopy procentowe", "stopy procentowe")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\himalayan\\cena opcji ~ stopy procentowe_himalayan_.svg"

# Korelacja

In [52]:
sims_correlation::Matrix{Float64} = correlation(1000,0.05,0.04,3,[10.0,10.0,10.0], 9.0, [0.07,0.07,0.07],[0.2,0.2,0.2])
plot_bar_plot(sims_correlation[:,6],"basic")
plot_bar_plot(sims_correlation[:,7],"antithetic")
plot_bar_plot(sims_correlation[:,8],"moment_matching")
plot_bar_plot(sims_correlation[:,9],"LHS")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\himalayan\\LHS_himalayan_.svg"